In [1]:
import pandas as pd
import numpy as np
import datetime

from linearmodels.panel import PanelOLS


### Data structure for panel regression
## bold text
Variables
1. id
2. wage_date
3. real_wage
4. is_after
5. intervention_index
6. age
7. gender
8. napa (yeshuv)
9. mosad_id
10. eshcol
11. group
12. leom

In [2]:
students = pd.read_csv("Data/Students data/students_processed_2.csv")
batla = pd.read_csv("Output/BONUS_HASAM_202206233_35.csv")

In [3]:
batla = batla.drop(columns=["Unnamed: 0"])

In [4]:
students["age"] = students["start_year"] - students["birth_year"]
students = students.loc[(students["end_year"]>=2016)]

In [5]:
keep_cols = ["id","gender","leom","yeshuv","mosad_id","end_date","eshcol","group","age"]
students = students[keep_cols]

In [6]:
df = pd.merge(batla, students, on="id", how="left")

In [7]:
df = df.loc[(df["gender"].isin(["ז","נ"]))]

In [8]:
df["date"] =  pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df["end_date"] =  pd.to_datetime(df["end_date"])

def inter(x):

    if x["date"].year == x["end_date"].year and x["date"].month == x["end_date"].month:
        return 1
    else:
        return 0

df["intervention"] = df.apply(inter, axis=1)

In [10]:
# Index monthes before and after intervention
df = df.sort_values(by=['id','date'], ascending=False)
df['intervention_index_pre'] = df.groupby('id',group_keys=False)['intervention'].apply(lambda x: (x - x.cumsum()).where(x == 1, lambda x: x.cumsum()))

df = df.sort_values(by=['id','date'], ascending=True)
df['intervention_index_post'] = df.groupby('id',group_keys=False)['intervention'].apply(lambda x: (x - x.cumsum()).where(x == 1, lambda x: x.cumsum()))
df["intervention_index_post"] = df["intervention_index_post"]*-1

def intervention(x):
    if x["intervention_index_pre"] < 0:
        return x["intervention_index_pre"]
    if x["intervention_index_pre"] == 0:
        return x["intervention_index_post"]
    else:
        return 0

df["intervention_index"] = df.apply(intervention, axis=1)
df= df.drop(columns=["intervention_index_pre","intervention_index_post"])

In [11]:
## Drop observation finished after batla data range
df = df.loc[(df["intervention"]!=0) | (df["intervention_index"]!=0)]

In [12]:
df["is_after"] = df.apply(lambda x: 1 if x["intervention_index"] > 0 else 0, axis=1)

In [15]:
cpi = {2014:99.8,2015:99.2,2016:98.7,2017:98.9,2018:99.7,2019:100.5,2020:100,2021:101.5,2022:105.9}

## Calculate the real wagebase 2021
base_year = 2021
def real_wage(x):
    if x["year"] in cpi.keys():
        return x["wage_per_month"]*(cpi[base_year]/cpi[x["year"]])
    else:
        return x["wage_per_month"]*(cpi[base_year]/cpi[base_year])

df["real_wage"] = df.apply(real_wage, axis=1)

In [19]:
df.head()

,id,wage_per_month,date,gender,leom,yeshuv,mosad_id,end_date,group,age,...,eshcol_עיבוד שבבי,eshcol_עיצוב שיער,eshcol_פיתוח תוכנה,eshcol_פרה רפואה,eshcol_צורפות/תכשיטנות,eshcol_צילום וסאונד,eshcol_קוסמטיקה,eshcol_קציני בטיחות בתעבורה,eshcol_שמאות רכב,"eshcol_תשתיות תקשורת- הדרכה, הטמעה ניהול ויישום"
43,30924,13999.83,2013-01-01,0,יהודי,3000.0,68061.0,2016-08-02,נהג אוטובוס ומונית,74.0,...,0,0,0,0,0,0,0,0,0,0
4404630,30924,13999.83,2013-02-01,0,יהודי,3000.0,68061.0,2016-08-02,נהג אוטובוס ומונית,74.0,...,0,0,0,0,0,0,0,0,0,0
8809217,30924,13999.83,2013-03-01,0,יהודי,3000.0,68061.0,2016-08-02,נהג אוטובוס ומונית,74.0,...,0,0,0,0,0,0,0,0,0,0
13213804,30924,13999.83,2013-04-01,0,יהודי,3000.0,68061.0,2016-08-02,נהג אוטובוס ומונית,74.0,...,0,0,0,0,0,0,0,0,0,0
17618391,30924,13999.83,2013-05-01,0,יהודי,3000.0,68061.0,2016-08-02,נהג אוטובוס ומונית,74.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
## Eshcol dummies get dummies
df = pd.get_dummies(df, columns=["eshcol"], prefix=["eshcol"])

## gender dummies
df["gender"] = df["gender"].apply(lambda x: 1 if x == "נ" else 0)

## mosad dummies
#df = pd.get_dummies(df, columns=["mosad_id"], prefix=["mosad"])

## yeshuv dummies
#df = pd.get_dummies(df, columns=["yeshuv"], prefix=["yeshuv"])

In [18]:
df.to_csv("Output/df_for_panel_fixedeffect.csv")